#  총 범죄 발생 건수 기점 분석

---------------------------

## 1. 테이블 명 및 용도 <br> 2. 데이터베이스 사용 세팅

--------------------------------------
- 범죄 유형 : crime_count_after(before)
- 범죄 지역 : crime_dosi_after(before)
- 범죄 장소 : crime_place_after(before)
- 범죄 요일 : crime_dotweek_after(before)
- 범죄 시간 : crime_time_after(before)
- 범죄 동기 : crime_motive_after(before)
- 범죄 나이 : crime_age_after(before)

In [1]:
import pandas as pd
import mysql.connector
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
aws_RDS = mysql.connector.connect(
host = "database-1.cz44qgu84nin.ap-northeast-2.rds.amazonaws.com",
user = "admin",
password = "123123123",
database = "edabase"
)

### 2. 1998년, 2008년 10% 증가 시각화

#### 1998 사용용도 별 데이터프레임 만들기

In [2]:
# 97,98,99 기점 근처 튀는값 중 범죄 분류별 소계값 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 중, 1994년, 1995년, 1996년, 1997년, 1998년, 1999년, 2000년, 2001년, 2002년, 2003년, 2004년, 2005년, 2006년, 2007년, 2008년, 2009년, 2010년\
                FROM crime_count_before2\
                where 중 in ('강력범', '절도범', '폭력범', '지능범', '풍속범', '기타형법범') and 소 = '소계'; ")

result = cursor.fetchall()

year98_totalcrime_type_df = pd.DataFrame(result)
year98_totalcrime_type_df
 

,중,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,강력범,12114,9762,10655,11599,13602,13400,14520,14858,14353,16519,15496,15357,16397,15973,17758,19790,25771
1,절도범,60255,62710,70238,83063,91438,89395,173876,180704,178457,187352,155311,188780,192670,212458,223216,256423,269410
2,폭력범,193047,186490,194891,200675,226409,282274,333630,338045,283930,294893,286464,285363,281969,295156,305508,315747,292347
3,지능범,158399,196845,227064,213445,253297,249769,192574,209364,220719,273867,290643,244680,258705,233213,260327,282106,268191
4,풍속범,16515,15815,15070,14722,15497,17655,19342,18519,16605,13668,13998,11492,19040,13563,14411,34793,15673
5,기타형법범,27571,25730,27260,29830,32992,37065,39179,41083,40541,40775,41223,45175,49944,58247,64451,71466,63447


In [4]:
meltyear98_totalcrime_type_df = pd.melt(year98_totalcrime_type_df, id_vars=['중'], var_name="년도")
meltyear98_totalcrime_type_df

,중,년도,value
0,강력범,1994년,12114
1,절도범,1994년,60255
2,폭력범,1994년,193047
3,지능범,1994년,158399
4,풍속범,1994년,16515
...,...,...,...
97,절도범,2010년,269410
98,폭력범,2010년,292347
99,지능범,2010년,268191
100,풍속범,2010년,15673


In [5]:
fig = px.bar(meltyear98_totalcrime_type_df, x='년도', y='value', color='중', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '중': 'Category'})

fig.update_layout(title='범죄 유형', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [6]:
# 97,98,99 기점 근처 튀는값 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 대, 소, 1997년, 1998년, 1999년\
                FROM crime_count_before2\
                where 대 = '총범죄'; ")

result = cursor.fetchall()

year98_total_df = pd.DataFrame(result)
year98_total_df
 

,대,소,1997년,1998년,1999년
0,총범죄,소계,1536652,1712233,1654064


#### 절도범 전후 그래프 시각화

In [7]:
# 97,98,99 기점 근처 튀는값 절도범 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT  소, 1994년, 1995년, 1996년, 1997년, 1998년, 1999년, 2000년, 2001년, 2002년, 2003년, 2004년, 2005년, 2006년, 2007년, 2008년, 2009년, 2010년\
                FROM crime_count_before2\
                where 중 = '절도범' and not 소 = '소계' and not 소 = '특가법(절도강간)'; ")

result = cursor.fetchall()

year98_theft_df = pd.DataFrame(result)
year98_theft_df
 

,소,1994년,1995년,1996년,1997년,1998년,1999년,2000년,2001년,2002년,2003년,2004년,2005년,2006년,2007년,2008년,2009년,2010년
0,절도,38787,43856,51853,62685,62749,60598,140388,147087,130527,144330,120952,156508,159902,171873,177429,193897,211991
1,특가법(절도),3879,2342,1996,2362,3812,7136,9388,12330,20014,19716,15546,13016,13644,16849,19633,27946,24459
2,야간주거침입절도,3037,2619,2528,2593,3202,3337,3510,3089,3199,3062,2243,2313,2436,2917,3527,5056,7437
3,야간주거침입절도(상습),2,4,2,0,12,7,4,9,3,11,2,45,10,1,9,18,42
4,특수절도,13891,13596,13653,14805,20870,17507,19620,17248,23155,18741,15488,16075,16029,19847,21527,27453,24049
5,자동차·선박등불법사용,0,0,0,351,379,343,384,373,425,452,400,382,337,348,314,311,253
6,특수절도(상습),17,16,5,20,67,41,12,56,45,129,14,0,41,9,21,79,24
7,상습절도,641,277,201,247,347,426,570,512,1089,911,666,441,271,614,756,1663,1155


In [8]:
dfyear98_theft_df = pd.melt(year98_theft_df, id_vars=['소'], var_name="년도")
dfyear98_theft_df

,소,년도,value
0,절도,1994년,38787
1,특가법(절도),1994년,3879
2,야간주거침입절도,1994년,3037
3,야간주거침입절도(상습),1994년,2
4,특수절도,1994년,13891
...,...,...,...
131,야간주거침입절도(상습),2010년,42
132,특수절도,2010년,24049
133,자동차·선박등불법사용,2010년,253
134,특수절도(상습),2010년,24


In [24]:
import plotly.express as px

fig = px.bar(dfyear98_theft_df, x='년도', y='value', color='소', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '소': 'Category'})

fig.update_layout(title='범죄 유형', xaxis_title='Year', yaxis_title='Value', )
fig.update_yaxes(range=[0, 250000])


fig.show()

In [15]:
# 97,98,99 기점 근처 튀는값 절도범 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT  소, 2011년, 2012년, 2013년, 2014년, 2015년, 2016년, 2017년, 2018년, 2019년, 2020년, 2021년, 2022년\
                FROM crime_count_after\
                where 중 = '절도범죄' and not 소 = '소계' and not 소 = '특가법(절도강간)'; ")

result = cursor.fetchall()

year22_theft_df = pd.DataFrame(result)
year22_theft_df
 

,소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,절도,241401,253166,248785,229194,211886,175765,160627,155723,165358,158512,148196,164143
1,절도(상습),0,0,1052,805,5017,3123,1940,1218,1070,971,785,697
2,특수절도,21730,21705,20362,17921,18636,15506,13128,11955,11783,11772,9142,9411
3,특수절도(상습),40,28,14,46,445,449,87,56,81,79,23,56
4,야간주거침입절도,5018,4553,5537,6292,7439,7065,6659,5994,6329,5465,5153,4697
5,야간주거침입절도(상습),16,14,15,20,333,276,132,89,122,105,109,32
6,불법사용,303,294,320,359,380,518,542,577,648,830,802,870
7,불법사용(상습),0,0,0,0,0,1,1,0,2,0,1,0
8,절도·불법사용(상습),599,365,0,0,0,0,0,0,0,0,0,0
9,특가법(절도),12255,10335,12257,11585,1717,334,641,1197,1564,1783,2198,2364


In [16]:
dfyear22_theft_df  = pd.melt(year22_theft_df, id_vars=['소'], var_name="년도")


fig = px.bar(dfyear22_theft_df, x='년도', y='value', color='소', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '소': 'Category'})

fig.update_layout(title='범죄 유형', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [10]:
# 97,98,99 기점 근처 튀는값 폭력범 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 소, 1997년, 1998년, 1999년\
                FROM crime_count_before2\
                where 중 = '폭력범' and not 소 = '소계' ; ")

result = cursor.fetchall()

year98_violent_df = pd.DataFrame(result)
year98_violent_df
 

,소,1997년,1998년,1999년
0,상해,22147,24765,29569
1,폭행,4607,7030,11077
2,협박,773,788,1101
3,공갈,1263,1169,1360


In [11]:
meltyear98_violent_df  = pd.melt(year98_violent_df, id_vars=['소'], var_name="년도")
fig = px.bar(meltyear98_violent_df, x='년도', y='value', color='소', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '소': 'Category'})

fig.update_layout(title='폭력범 범죄 유형', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [12]:
# 97,98,99 기점 근처 튀는값 지능범 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 소, 1997년, 1998년, 1999년\
                FROM crime_count_before2\
                where 중 = '지능범' and not 소 = '소계' ; ")

result = cursor.fetchall()

year98_it_df = pd.DataFrame(result)
year98_it_df
 

,소,1997년,1998년,1999년
0,사기,176588,206702,190944
1,횡령,12929,16536,14232
2,배임,3066,3762,3933


In [13]:
meltyear98_it_df  = pd.melt(year98_it_df, id_vars=['소'], var_name="년도")
fig = px.bar(meltyear98_it_df, x='년도', y='value', color='소', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '소': 'Category'})

fig.update_layout(title='지능범 세부 범죄', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [14]:
# 97,98,99 기점 근처 튀는값 강력범 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT  소, 1997년, 1998년, 1999년\
                FROM crime_count_before2\
                where 중 = '강력범' and not 소 = '소계' ; ")

result = cursor.fetchall()

year98_power_df = pd.DataFrame(result)
year98_power_df
 

,소,1997년,1998년,1999년
0,살인,784,963,976
1,강도,4420,5516,4972
2,강간,5627,5978,6359
3,방화,768,1145,1093


In [15]:
meltyear98_power_df  = pd.melt(year98_power_df, id_vars=['소'], var_name="년도")
fig = px.bar(meltyear98_power_df, x='년도', y='value', color='소', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '소': 'Category'})

fig.update_layout(title='강력범 세부 범죄', xaxis_title='Year', yaxis_title='Value')

fig.show()

#### 2008 사용용도 별 데이터프레임 만들기

In [16]:
# 08년도 교통범 값 만들기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 소 as 중, sum(2007년) 2007년, sum(2008년) 2008년,sum(2009년) 2009년\
                FROM crime_count_before2\
                where 소 like '%교통%'; ")

result = cursor.fetchall()

check = pd.DataFrame(result)
check
 

,중,2007년,2008년,2009년
0,교통 방해의 죄,697119,787022,632690


In [17]:
# 08년도 교통범 값 만들기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 소 as 중, sum(2007년) 2007년, sum(2008년) 2008년,sum(2009년) 2009년\
                FROM crime_count_before2\
                where 소 like '%교통%'; ")

result = cursor.fetchall()

check = pd.DataFrame(result)
check
 

,중,2007년,2008년,2009년
0,교통 방해의 죄,697119,787022,632690


In [18]:
# 07,08,09 기점 근처 튀는값 중 범죄 분류별 소계값 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 중, 2007년, 2008년, 2009년\
                FROM crime_count_before2\
                where 중 in ('강력범', '절도범', '폭력범', '지능범', '풍속범') and 소 = '소계'; ")

result = cursor.fetchall()

year08_totalcrime_type_df = pd.DataFrame(result)
year08_totalcrime_type_df = pd.concat([check, year08_totalcrime_type_df])
year08_totalcrime_type_df.reset_index(inplace=True, drop=True)
year08_totalcrime_type_df.loc[0, '중'] = '교통범'
year08_totalcrime_type_df
 

,중,2007년,2008년,2009년
0,교통범,697119,787022,632690
1,강력범,15973,17758,19790
2,절도범,212458,223216,256423
3,폭력범,295156,305508,315747
4,지능범,233213,260327,282106
5,풍속범,13563,14411,34793


In [19]:
import pandas as pd
import plotly.express as px
import numpy as np

# Assuming year08_totalcrime_type_df is already defined and loaded

# Melt the DataFrame
meltyear08_totalcrime_type_df = pd.melt(year08_totalcrime_type_df, id_vars=['중'], var_name="년도")

fig = px.line(meltyear08_totalcrime_type_df, 
              x='년도', 
              y='value', 
              color='중', 
              line_group='중',  
              labels={'년도': 'Year', 'value': 'Value', '중': 'Category'})

fig.update_layout(title='2007~2008년 교통, 강력, 절도, 폭력, 지능, 풍속 (Logarithmic Scale)', 
                  xaxis_title='Year', 
                  yaxis_title='Log Value')

fig.show()


2008년에 교통범은 줄어들고 풍속범은 증가하는것을 볼수있다.

In [20]:

meltyear08_totalcrime_type_df = pd.melt(year08_totalcrime_type_df, id_vars=['중'], var_name="년도")
fig = px.bar(meltyear08_totalcrime_type_df, x='년도', y='value', color='중', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '중': 'Category'})

fig.update_layout(title='2007~2008년 교통, 강력, 절도, 폭력, 지능, 풍속', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [21]:
# 07,08,09 기점 근처 튀는값 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 대, 소, 2007년, 2008년, 2009년\
                FROM crime_count_before2\
                where 대 = '총범죄'; ")

result = cursor.fetchall()

year08_total_df = pd.DataFrame(result)
year08_total_df
 

,대,소,2007년,2008년,2009년
0,총범죄,소계,1836496,2063737,2020209


In [22]:
# 07,08,09  기점 근처 튀는값 절도범 총 범죄 건수 가지고 오기

cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT  소, 2007년, 2008년, 2009년\
                FROM crime_count_before2\
                where 중 = '절도범' and not 소 = '소계' and not 소 = '특가법(절도강간)'; ")

result = cursor.fetchall()

year08_theft_df = pd.DataFrame(result)
year08_theft_df
 

,소,2007년,2008년,2009년
0,절도,171873,177429,193897
1,특가법(절도),16849,19633,27946
2,야간주거침입절도,2917,3527,5056
3,야간주거침입절도(상습),1,9,18
4,특수절도,19847,21527,27453
5,자동차·선박등불법사용,348,314,311
6,특수절도(상습),9,21,79
7,상습절도,614,756,1663


In [23]:
melt_year08_theft_df = pd.melt(year08_theft_df, id_vars=['소'], var_name="년도")
fig = px.bar(melt_year08_theft_df, x='년도', y='value', color='소', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '소': 'Category'})

fig.update_layout(title='절도범 세부 범죄', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [24]:
cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 소, 2007년, 2008년, 2009년\
                FROM crime_count_before2\
                where 중 = '폭력범' and not 소 = '소계' ; ")

result = cursor.fetchall()

year08_violent_df = pd.DataFrame(result)
year08_violent_df
 

,소,2007년,2008년,2009년
0,상해,0,0,0
1,폭행,0,0,0
2,협박,0,0,0
3,공갈,0,0,0


In [25]:
cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 소, 2007년, 2008년, 2009년\
                FROM crime_count_before2\
                where 중 = '지능범' and not 소 = '소계' ; ")

result = cursor.fetchall()

year08_it_df = pd.DataFrame(result)
year08_it_df
 

,소,2007년,2008년,2009년
0,사기,0,0,0
1,횡령,0,0,0
2,배임,0,0,0


In [26]:
cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 소, 2007년, 2008년, 2009년\
                FROM crime_count_before2\
                where 중 = '강력범' and not 소 = '소계' ; ")

result = cursor.fetchall()

year08_power_df = pd.DataFrame(result)
year08_power_df
 

,소,2007년,2008년,2009년
0,살인,1111,1109,1374
1,강도,4439,4811,6351
2,강간,8726,9883,10192
3,방화,1697,1955,1873


In [27]:
meltyear08_power_df  = pd.melt(year08_power_df, id_vars=['소'], var_name="년도")
fig = px.bar(meltyear08_power_df, x='년도', y='value', color='소', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '소': 'Category'})

fig.update_layout(title='강력범 범죄 유형', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [28]:
winddf= pd.read_csv("../Data/범죄발생및검거현황/풍속범(11~22).csv", encoding="utf-8")
winddf
 

,시점,성풍속범죄,도박범죄
0,2011,13381,15453
1,2012,14935,9866
2,2013,19385,7988
3,2014,16683,8293
4,2015,16219,8272
5,2016,12719,13446
6,2017,13812,8689
7,2018,14158,6004
8,2019,13145,8008
9,2020,14497,8135


In [29]:
winddf2= pd.read_csv("../Data/범죄발생및검거현황/풍속범(94~10).csv", encoding="utf-8")
winddf2 = winddf2[11:]

#### 도박범죄 발생이 늘어난것을 볼수있다.
경제적 스트레스와 불확실성: 미국에서 시작된 서브프라임 모기지 사태로 인한 글로벌 금융 위기는 한국을 포함한 전 세계에 경제적 불안정을 초래했습니다. 많은 사람들이 일자리 상실, 소득 감소, 경제적 불안감 등 재정적 어려움에 직면하게 되었습니다. 이러한 시기에 일부 사람들은 재정 문제를 해결하거나 도피하기 위한 방법으로 도박을 선택할 수 있습니다.

심리적 요인: 경제 불황은 스트레스, 불안, 절망감을 증가시킬 수 있습니다. 도박은 이러한 심리적 압박을 해소하거나 일시적인 도피처를 제공할 수 있습니다. 특히 다른 재정적 수단이 어려운 상황에서 큰돈을 딸 수 있다는 희망은 더욱 매력적일 수 있습니다.

도박 기회의 증가: 경제 침체기에는 불법 도박 활동이 증가하는 경향이 있습니다. 운영자들은 증가된 수요를 이용하며, 당국이 경제 위기와 관련된 다른 시급한 문제에 집중하면서 도박 법률의 집행이 느슨해질 수 있습니다.

문화적 요인: 한국을 포함한 많은 문화권에서 도박은 경계와 매혹을 동시에 불러일으킵니다. 경제적 어려움이 닥치면 도박에 대한 문화적 태도가 변화할 수 있으며, 이를 통해 도박이 더 사회적으로 용인되거나 심지어 재정적 구제 수단으로 여겨질 수 있습니다.

결론적으로, 2008년에서 2009년 사이 한국에서 도박 범죄의 급증은 서브프라임 모기지 사태로 인한 광범위한 경제적 고통의 결과로 볼 수 있습니다. 재정적 어려움, 심리적 스트레스, 도박 기회의 증가, 문화적 태도, 규제의 허점, 미디어 영향 등이 모두 이러한 현상에 기여했습니다.

In [30]:
meltwinddf = pd.melt(winddf2, id_vars=['시점'], var_name="유형", value_name="value")


fig = px.bar(meltwinddf, x='시점', y='value', color='유형', 
             barmode='group',
             text='value', 
             labels={'시점': 'Year', 'value': 'Value', '유형': '풍속범죄 유형'})


fig.update_layout(
    title='풍속범 범죄 유형', 
    xaxis_title='Year', 
    yaxis_title='Value',
    xaxis=dict(
        tickmode='linear',
        tick0=2011,
        dtick=1
    )
)


fig.show()

##### 11년 ~ 22년 충속범 세부 범죄 바차트

In [31]:
meltwinddf = pd.melt(winddf, id_vars=['시점'], var_name="유형", value_name="value")


fig = px.bar(meltwinddf, x='시점', y='value', color='유형', 
             barmode='group',
             text='value', 
             labels={'시점': 'Year', 'value': 'Value', '유형': '풍속범죄 유형'})


fig.update_layout(
    title='풍속범 범죄 유형', 
    xaxis_title='Year', 
    yaxis_title='Value',
    xaxis=dict(
        tickmode='linear',
        tick0=2011,
        dtick=1
    )
)


fig.show()

### 2. 2005년, 2010년, 2017년 -10% 감소 시각화 

In [32]:
cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 중, 2004년, 2005년, 2006년, 2009년,2010년\
                FROM crime_count_before2\
                where 중 in ('강력범', '절도범', '폭력범', '지능범', '풍속범', '기타형법범') and 소 = '소계'; ")

result = cursor.fetchall()

part2_totalcrime_type_df = pd.DataFrame(result)
part2_totalcrime_type_df
 

,중,2004년,2005년,2006년,2009년,2010년
0,강력범,15496,15357,16397,19790,25771
1,절도범,155311,188780,192670,256423,269410
2,폭력범,286464,285363,281969,315747,292347
3,지능범,290643,244680,258705,282106,268191
4,풍속범,13998,11492,19040,34793,15673
5,기타형법범,41223,45175,49944,71466,63447


In [33]:
meltpart2_totalcrime_type_df = pd.melt(part2_totalcrime_type_df, id_vars=['중'], var_name="년도")
fig = px.bar(meltpart2_totalcrime_type_df, x='년도', y='value', color='중', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', 'value': 'Value', '중': 'Category'})

fig.update_layout(title='범죄 유형', xaxis_title='Year', yaxis_title='Value')

fig.show()

In [34]:
cursor = aws_RDS.cursor(dictionary=True ,buffered=True)
cursor.execute("SELECT 중, 2011년, 2016년, 2017년, 2018년\
                FROM crime_count_after\
                where 중 in ('강력범죄', '절도범죄', '폭력범죄', '지능범죄', '풍속범죄') and 소 = '소계'; ")

result = cursor.fetchall()

part3_totalcrime_type_df = pd.DataFrame(result)
part3_totalcrime_type_df['중'] = part3_totalcrime_type_df['중'].str.replace('죄', '')
part3_totalcrime_type_df
 

,중,2011년,2016년,2017년,2018년
0,강력범,26699,25765,27274,26787
1,절도범,281362,203037,183757,176809
2,폭력범,311945,309394,293086,287611
3,지능범,279802,312577,302466,344698
4,풍속범,28834,26165,22501,20162


In [35]:
 # DataFrame을 melt하여 긴 형식으로 변환
melted_part2 = part2_totalcrime_type_df.melt(id_vars=['중'], var_name='년도', value_name='값')
melted_part3 = part3_totalcrime_type_df.melt(id_vars=['중'], var_name='년도', value_name='값')

melted_part2, melted_part3
# 두 DataFrame 결합
totalminusdf= pd.concat([melted_part2, melted_part3])

In [36]:
totalminusdf.reset_index(inplace=True, drop=True)
totalminusdf

,중,년도,값
0,강력범,2004년,15496
1,절도범,2004년,155311
2,폭력범,2004년,286464
3,지능범,2004년,290643
4,풍속범,2004년,13998
5,기타형법범,2004년,41223
6,강력범,2005년,15357
7,절도범,2005년,188780
8,폭력범,2005년,285363
9,지능범,2005년,244680


In [37]:

fig = px.bar(totalminusdf, x='년도', y='값', color='중', 
             barmode='group',
             text='년도', 
             labels={'년도': 'Year', '값': 'Value', '중': 'Category'})

fig.update_layout(title='범죄 유형', xaxis_title='Year', yaxis_title='Value')

fig.show()

* 위를 봐서는 2005년, 2010년, 2017년에 전년대비 -10%이상 줄어든 이유를 알기힘들다.

#### 사용용도 별 데이터프레임 만들기

In [38]:
import json

cities = [
    {"name": "서울", "lat": 37.5665, "lon": 126.9780},
    {"name": "부산", "lat": 35.1796, "lon": 129.0756},
    {"name": "대구", "lat": 35.8714, "lon": 128.6014},
    {"name": "인천", "lat": 37.4563, "lon": 126.7052},
    {"name": "광주", "lat": 35.1595, "lon": 126.8526},
    {"name": "대전", "lat": 36.3504, "lon": 127.3845},
    {"name": "울산", "lat": 35.5384, "lon": 129.3114},
    {"name": "세종", "lat": 36.4800, "lon": 127.2890},
    {"name": "수원시", "lat": 37.2636, "lon": 127.0286},
    {"name": "성남시", "lat": 37.4386, "lon": 127.1378},
    {"name": "고양시", "lat": 37.6583, "lon": 126.8320},
    {"name": "용인시", "lat": 37.2410, "lon": 127.1775},
    {"name": "부천시", "lat": 37.5035, "lon": 126.7665},
    {"name": "안산시", "lat": 37.3219, "lon": 126.8309},
    {"name": "안양시", "lat": 37.3943, "lon": 126.9568},
    {"name": "남양주시", "lat": 37.6360, "lon": 127.2165},
    {"name": "화성시", "lat": 37.1995, "lon": 126.8312},
    {"name": "평택시", "lat": 36.9925, "lon": 127.1128},
    {"name": "의정부시", "lat": 37.7380, "lon": 127.0337},
    {"name": "시흥시", "lat": 37.3799, "lon": 126.8032},
    {"name": "춘천시", "lat": 37.8813, "lon": 127.7300},
    {"name": "원주시", "lat": 37.3422, "lon": 127.9200},
    {"name": "강릉시", "lat": 37.7519, "lon": 128.8760},
    {"name": "동해시", "lat": 37.5247, "lon": 129.1142},
    {"name": "태백시", "lat": 37.1665, "lon": 128.9886},
    {"name": "속초시", "lat": 38.2070, "lon": 128.5918},
    {"name": "삼척시", "lat": 37.4449, "lon": 129.1650},
    {"name": "청주시", "lat": 36.6424, "lon": 127.4890},
    {"name": "충주시", "lat": 36.9911, "lon": 127.9259},
    {"name": "제천시", "lat": 37.1323, "lon": 128.1910},
    {"name": "천안시", "lat": 36.8151, "lon": 127.1135},
    {"name": "공주시", "lat": 36.4469, "lon": 127.1190},
    {"name": "보령시", "lat": 36.3494, "lon": 126.6032},
    {"name": "아산시", "lat": 36.7900, "lon": 127.0017},
    {"name": "서산시", "lat": 36.7850, "lon": 126.4503},
    {"name": "논산시", "lat": 36.1873, "lon": 127.0987},
    {"name": "계룡시", "lat": 36.2746, "lon": 127.2487},
    {"name": "당진시", "lat": 36.8899, "lon": 126.6461},
    {"name": "전주시", "lat": 35.8242, "lon": 127.1480},
    {"name": "군산시", "lat": 35.9676, "lon": 126.7366},
    {"name": "익산시", "lat": 35.9483, "lon": 126.9576},
    {"name": "정읍시", "lat": 35.5698, "lon": 126.8558},
    {"name": "남원시", "lat": 35.4164, "lon": 127.3906},
    {"name": "김제시", "lat": 35.8038, "lon": 126.8805},
    {"name": "목포시", "lat": 34.8124, "lon": 126.3922},
    {"name": "여수시", "lat": 34.7604, "lon": 127.6622},
    {"name": "순천시", "lat": 34.9506, "lon": 127.4872},
    {"name": "나주시", "lat": 35.0159, "lon": 126.7109},
    {"name": "광양시", "lat": 34.9406, "lon": 127.6958},
    {"name": "포항시", "lat": 36.0190, "lon": 129.3435},
    {"name": "경주시", "lat": 35.8562, "lon": 129.2249},
    {"name": "김천시", "lat": 36.1398, "lon": 128.1135},
    {"name": "안동시", "lat": 36.5684, "lon": 128.7294},
    {"name": "구미시", "lat": 36.1196, "lon": 128.3445},
    {"name": "영주시", "lat": 36.8058, "lon": 128.6242},
    {"name": "영천시", "lat": 35.9733, "lon": 128.9387},
    {"name": "상주시", "lat": 36.4109, "lon": 128.1589},
    {"name": "문경시", "lat": 36.5860, "lon": 128.1870},
    {"name": "경산시", "lat": 35.8252, "lon": 128.7414},
    {"name": "창원시", "lat": 35.2280, "lon": 128.6811},
    {"name": "진주시", "lat": 35.1795, "lon": 128.1076},
    {"name": "통영시", "lat": 34.8544, "lon": 128.4332},
    {"name": "사천시", "lat": 35.0038, "lon": 128.0642},
    {"name": "김해시", "lat": 35.2281, "lon": 128.8812},
    {"name": "밀양시", "lat": 35.5038, "lon": 128.7464},
    {"name": "거제시", "lat": 34.8804, "lon": 128.6211},
    {"name": "양산시", "lat": 35.3350, "lon": 129.0370},
    {"name": "제주시", "lat": 33.4996, "lon": 126.5312},
    {"name": "서귀포시", "lat": 33.2539, "lon": 126.5600}
]

def create_polygon(lat, lon, size=0.05):
    return [
        [lon - size, lat - size],
        [lon + size, lat - size],
        [lon + size, lat + size],
        [lon - size, lat + size],
        [lon - size, lat - size]
    ]

geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {"name": city["name"]},
            "geometry": {
                "type": "Polygon",
                "coordinates": [create_polygon(city["lat"], city["lon"])]
            }
        } for city in cities
    ]
}

with open('korean_cities_polygons.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("GeoJSON file has been created: korean_cities_polygons.geojson")

GeoJSON file has been created: korean_cities_polygons.geojson


In [39]:
import folium

m = folium.Map(location=[36.5, 127.5], zoom_start=7)

folium.GeoJson(
    'korean_cities_polygons.geojson',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'black',
        'weight': 6,
        'fillOpacity': 0.8,
    },
    tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['도시:'])
).add_to(m)

m